<a href="https://colab.research.google.com/github/Dutta-SD/NLP/blob/master/Aggression_Detection/Experiments/Aggression_Detection_Normal_ML_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggression Experiments

## CODE NOT FULLY FUNCTIONAL. LAST CELL IS UNDER CONSTRUCTION

In [69]:
import pandas as pd
import numpy as np
from sklearn import metrics, ensemble, svm, feature_extraction, naive_bayes, neural_network
import xgboost
from sklearn.model_selection import cross_val_score
import re
import nltk

In [11]:
TRAIN_URL_TASK_1 = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/Aug_Data_Aggression/TASK_A_train_aug_english.csv'
TRAIN_URL_TASK_2 = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/Aug_Data_Aggression/TASK_B_train_aug_english.csv'

VAL_URL = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/trac2_eng_dev.csv'

# Task Description
* **A** - Aggression detection
* **B** - Misogyny Detection

In [12]:
# task 1 - Aggression
train = pd.read_csv(TRAIN_URL_TASK_1)
val = pd.read_csv(VAL_URL)
train.head()

,Text,Sub-task A
0,Next part,NAG
1,Iii8mllllllm\nMdxfvb8o90lplppi0005,NAG
2,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,NAG
3,What the fuck was this? I respect shwetabh and...,NAG
4,Concerned authorities should bring arundathi R...,NAG


In [13]:
def seed_all():
  np.random.seed(0)
  
seed_all()

In [71]:
def clean_one_text(text):
    # Cleans one text and returns it
    
    # Clean Punctuation
    # Might remove emojis too
    # 
    res = re.sub(r'[^\w\s]', '', text)
    tk = nltk.TweetTokenizer()
    stmr = nltk.stem.snowball.SnowballStemmer("english")
    clean_tokens = [stmr.stem(token) for token in tk.tokenize(res)]
    return ' '.join(clean_tokens)

In [83]:
def get_clean_dataset(
    df_raw,
    target_mapping,
    train = True,
    task_name='A', 
    vectorizer=None,
    string_cleaner=None,
    seed = 0):
  '''
  ===============================================================
  get_clean_dataset - cleans the dataset, returns text and labels
  ===============================================================

  :df_raw - pandas dataframe for cleaning
  :target_mapping - map for the targets
  :train - flag to see if training data sent or not
  :task_name - the target to predict
  :vectorizer - vectorizes the data
  :string_cleaner - useful for removing punctuation, etc(function)
  '''

  col_str = f'Sub-task {task_name}'

  if 'ID' in df_raw.columns:
    df_raw = df_raw.drop(['ID'], axis = 1)

  targets = df_raw[col_str].map(target_mapping).values
  text = df_raw['Text'].values.astype('str')

  if string_cleaner is not None:
    v_cleaner = np.vectorize(string_cleaner)
    text = v_cleaner(text)

  if vectorizer:
    if train:
      text = vectorizer.fit_transform(text)
    else:
      text = vectorizer.transform(text)  

  return text, targets

In [85]:
X_train, y_train = get_clean_dataset(
    df_raw = train,
    target_mapping = task_1_map,
    train=True,
    task_name = 'A',
    vectorizer = v2,
    string_cleaner = clean_one_text
    
)
X_val, y_val = get_clean_dataset(
    df_raw = val,
    target_mapping = task_1_map,
    train=False,
    task_name = 'A',
    vectorizer = v2,
    string_cleaner = clean_one_text
)

In [95]:
task_1_map ={
    'NAG' : 0,
    'CAG' : 1,
    'OAG' : 2
}

# count of objects
s = np.bincount(train.iloc[ : , 1].map(task_1_map))

# Vectorizers
v1 = feature_extraction.text.CountVectorizer()
v2 = feature_extraction.text.TfidfVectorizer(strip_accents='unicode')

weights = {i : s.sum() / s[i] for i in range(len(s))}

model1 = ensemble.RandomForestClassifier(max_depth = 3,
                                         class_weight=weights, random_state=0, criterion='entropy')
model2 = svm.LinearSVC(C = 0.5, class_weight=weights, random_state=0)
model3 = xgboost.XGBClassifier(max_depth = 10, gamma = 0.1 ,scale_pos_weight=weights, random_state=0)
model4 = neural_network.MLPClassifier(hidden_layer_sizes = (128, 1024, 128),
                                      random_state=0, 
                                      verbose=True, 
                                      learning_rate='adaptive', 
                                      max_iter=20,
                                      tol=1e-3)

model_list = [
              model1, 
              model2, 
              model3, 
            #   model4,
              ]

In [96]:
# Training Loop -- TASK A
print(f"Vectorizer used : {type(v2).__name__}", end="\n\n")

for i, model in enumerate(model_list):
    print("-"*80)
    print(f"MODEL NO : {i}, TRAINING STARTS...")
    print(f"MODEL NAME : {type(model).__name__}")
    print(f"MODEL RANDOM SEED(IF ANY) : {model.random_state}")

    # Fitting
    preds = model.fit(X_train, y_train).predict(X_val)
    true_preds = y_val

    # Validation
    print(metrics.classification_report(true_preds, preds))
    print(metrics.confusion_matrix(true_preds, preds))

    print(f"\nDone with model {i}")

Vectorizer used : TfidfVectorizer

--------------------------------------------------------------------------------
MODEL NO : 0, TRAINING STARTS...
MODEL NAME : RandomForestClassifier
MODEL RANDOM SEED(IF ANY) : 0
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       836
           1       0.34      0.38      0.36       117
           2       0.34      0.32      0.33       113

    accuracy                           0.75      1066
   macro avg       0.52      0.52      0.52      1066
weighted avg       0.75      0.75      0.75      1066

[[721  61  54]
 [ 57  44  16]
 [ 54  23  36]]

Done with model 0
--------------------------------------------------------------------------------
MODEL NO : 1, TRAINING STARTS...
MODEL NAME : LinearSVC
MODEL RANDOM SEED(IF ANY) : 0
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       836
           1       0.31      0.27      0.29       117
         

In [18]:
# '''
# DO NOT RUN THIS CELL. UNDER CONSTRUCTION
# '''

# task_2_map ={
#     'NGEN' : 0,
#     'GEN' : 1,
# }
# v1 = feature_extraction.text.CountVectorizer()
# v2 = feature_extraction.text.TfidfVectorizer()

# train_clean = get_clean_dataset(train, True,'B','A', task_2_map, vectorizer=v2)
# val_clean = get_clean_dataset(val, False, 'B', 'A', task_2_map, vectorizer=v2)

# # print(train_clean[1][1:10])
# s_2 = np.bincount(train_clean[1])
# # print(s_2)

# weights_2 = {i : s_2.sum() / s_2[i] for i in range(2)}
# w = s_2[0]/s_2[1]

# model1 = ensemble.RandomForestClassifier(class_weight=weights_2, random_state=0, criterion='entropy')
# model2 = svm.LinearSVC(class_weight=weights_2, random_state=0)
# model3 = xgboost.XGBClassifier(scale_pos_weight=w, random_state=0)
# model4 = neural_network.MLPClassifier(random_state=0, verbose=True, learning_rate='adaptive',max_iter=5 )

# model_list = [model1, model2, model3, model4]

# print(f"Vectorizer used : {type(v2).__name__}", end="\n\n")

# for i, model in enumerate(model_list):
#   print(f"model no {i}, training")
#   print(f"model name {type(model).__name__}")
#   preds = model.fit(train_clean[0], train_clean[1]).predict(val_clean[0])
#   true_preds = val_clean[1]
#   print(metrics.classification_report(true_preds, preds))
#   print(f"\nDone with model {i}")